In [1]:
import json
import openpyxl

# Load the workbook
wb = openpyxl.load_workbook('shop_site.xlsx')

comments_of_shops = {}
# Iterate over each sheet
for sheet_name in wb.sheetnames:
    if sheet_name == 'Sheet1':
        continue
    # Select the sheet
    sheet = wb[sheet_name]

    comments = []

    # Get the row values (assuming first row is title)
    titles = [cell.value for cell in sheet[1]]

    # Iterate over each row starting from 2nd row (assuming first row is title)
    for row_num in range(2, sheet.max_row + 1):
        comment = {}
        # Get the row values
        row_values = [cell.value for cell in sheet[row_num]]
        for i in range(len(titles)):
            comment[titles[i]] = row_values[i]
        comments.append(comment)
    comments_of_shops[sheet_name] = comments

with open('comments_of_shops.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(comments_of_shops, ensure_ascii=False))


In [2]:
import json
print(open('comments_of_shops.json','r').read())
comments_of_shops = json.loads(open('comments_of_shops.json','r').read())

{"再现山海经·大型沉浸式体验馆": [{"UserName": "匿名用户", "UserLink": "", "UserAvatar": "http://p1.meituan.net/dpgroup/40d654f85e4ca6583db6c141873edfd129040.png", "Score": "", "Content": "商家宣传内容，远超实际体验，一共造景体验的不超过6个房间，且都是格子间展示。山海经中的大部分角色就只有5分钟的一个3D影片呈现，其他展示内容非常简单，适合1-2年级的孩子看，太小了看不懂，大点的娃娃觉得被欺骗，太单一。", "Images": "https://qcloud.dpfile.com/pc/yzm09KNes_SH-1AHQLtJJRJA-ROKsuJ0iWHQLHT29AA85m6KLxJrZMuRwDiePBFkjoJrvItByyS4HHaWdXyO_I7F0UeCRQYMHlogzbt7GHgNNiIYVnHvzugZCuBITtvjski7YaLlHpkrQUr5euoQrg.jpg, https://qcloud.dpfile.com/pc/V4AJ7msJ_nNR9ni_XonjU8WRQpG_OBkVPIRCFzT4Atax0FB5XAwjlC-OyK5Y2SgajoJrvItByyS4HHaWdXyO_I7F0UeCRQYMHlogzbt7GHgNNiIYVnHvzugZCuBITtvjski7YaLlHpkrQUr5euoQrg.jpg, https://qcloud.dpfile.com/pc/e3-ShHmahkUu1A1wo88YI_U4TkhDNOa96JdXr_BxewYHo8itsBkcmVncTI8kIYxjjoJrvItByyS4HHaWdXyO_I7F0UeCRQYMHlogzbt7GHgNNiIYVnHvzugZCuBITtvjski7YaLlHpkrQUr5euoQrg.jpg, https://qcloud.dpfile.com/pc/U2ogYsQtIBmxTrex5C7zdX9FxXTs1JlsqRaOm6cDK7jj9dg9E8SMub24UXI9Ivf5joJrvItByyS4HHaWdXyO_I7F0UeCRQYMHlogzbt7GHgNNiIYVnHvzugZCu

In [57]:
from ollama_chat import chat_reponse
llama3 = 'llama3'
gemma2 = 'gemma2:9b-instruct-q6_K'
qwen2 = 'qwen2:7b-instruct-q6_K'
mixtral = 'mixtral:8x7b-instruct-v0.1-q6_K'
for shop in comments_of_shops:
    comments = comments_of_shops[shop]
    for comment in comments:
        text = comment.get('Content')
        if not text:
            continue
        message = \
        """用户将输入来自某沉浸式艺术展的用户评论，请完整地阅读评论并对它进行分析。
        按照下面的五个方面对他进行打分，每一方面总分为五分，只能打整数分。
        分数越高说明这个评价包含的评价越符合这一个方面的描述。
        如果没有相应的评价或者无法推测评论者的态度，则评零分。
        评判的五个方面分别是：
        1. 性价比：消费者对于性价比有负面评价，这种评价来源于期望与实际体验之间的不匹配
        2. 沉浸体验：消费者对于付费进行沉浸体验本身的轻视、不尊重或不认可
        3. 体验引导：消费者常因体验的缺乏引导性而产生迷失感，对整体体验感到不满
        4. 后续消费/反馈：消费者在体验结束后仍对体验持续性有更高期待，认为缺少后续环节如周边，纪念品，收集品，信息交流渠道等
        5. 展览内容：消费者对于沉浸展览的主题内容有负面评价，认为主题内容难以接受，例如太抽象、无法理解、看不懂等
        ###
        在进行回复时，请严格按照以下格式：
        <分数一> <分数二> <分数三> <分数四> <分数五>
        ###
        不要在回复中添加任何的前缀！
        不需要在回复中包含评论理由！
        """
        review = chat_reponse(text, model=gemma2, system_prompt=message)
        print(text)
        print(review)
        break

商家宣传内容，远超实际体验，一共造景体验的不超过6个房间，且都是格子间展示。山海经中的大部分角色就只有5分钟的一个3D影片呈现，其他展示内容非常简单，适合1-2年级的孩子看，太小了看不懂，大点的娃娃觉得被欺骗，太单一。
{'model': 'gemma2:9b-instruct-q6_K', 'created_at': '2024-07-23T06:32:43.1282456Z', 'message': {'role': 'assistant', 'content': '5 1 3 4 4 \n'}, 'done_reason': 'stop', 'done': True, 'total_duration': 21002434600, 'load_duration': 7831312100, 'prompt_eval_count': 397, 'prompt_eval_duration': 12028708000, 'eval_count': 12, 'eval_duration': 1136921000}
